In [12]:
!pip install -q transformers evaluate sentencepiece
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.3 MB/s eta 0:00:00


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

CUDA available: True
Device: Tesla T4


In [2]:
from google.colab import files
uploaded = files.upload()   # 파일 선택 창이 뜹니다.
file_path = list(uploaded.keys())[0]
print("업로드된 파일:", file_path)

Saving 순화표현모델 데이터_리라이팅완료.jsonl to 순화표현모델 데이터_리라이팅완료.jsonl
업로드된 파일: 순화표현모델 데이터_리라이팅완료.jsonl


In [5]:
import json
import re
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import evaluate
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 1) 모델 & 토크나이저 로드 (KoT5-base) → GPU로 이동
model_name = "wisenut-nlp-team/KoT5-base"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [6]:
# 1) JSONL 파일 로드
with open(file_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f if line.strip()]
print(f"총 샘플 수: {len(raw_data)}")

# 2) 전처리 함수 정의
def preprocess(example):
    context_lines = example["context"][:-1]
    toxic_line    = example["context"][-1]
    prefix = "rewrite last line politely:"       # 영어 프롬프트
    full_input = " ".join([prefix] + context_lines + [toxic_line])
    return {"input_text": full_input, "target_text": example["output"]}

# 3) 전처리 적용
processed_data = [preprocess(d) for d in raw_data]
print("예시 입력/출력:")
print(processed_data[0])

총 샘플 수: 3100
예시 입력/출력:
{'input_text': 'rewrite last line politely: A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴;; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴', 'target_text': 'B: 수학쌤 목소리 너무 지루하긴 해'}


In [7]:
from torch.utils.data import Dataset
from transformers import DataCollatorForSeq2Seq

# 1) RewriteDataset 클래스 (pad → -100 마스킹, GPU 이동은 Trainer 내부에서 자동 처리)
class RewriteDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data       = data
        self.tokenizer  = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # 입력 인코딩
        enc = self.tokenizer(
            item["input_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        # 레이블 인코딩
        lbl = self.tokenizer(
            text_target=item["target_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze()
        # pad → -100
        lbl[lbl == self.tokenizer.pad_token_id] = -100

        # batch 차원 제거
        batch = {k: v.squeeze() for k, v in enc.items()}
        batch["labels"] = lbl
        return batch

# 2) train/test 분할
train_data, test_data = train_test_split(processed_data, test_size=0.1, random_state=42)
train_dataset = RewriteDataset(train_data, tokenizer)
test_dataset  = RewriteDataset(test_data, tokenizer)

# 3) dynamic padding collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

print(f"학습 샘플 수: {len(train_dataset)}, 검증 샘플 수: {len(test_dataset)}")

학습 샘플 수: 2790, 검증 샘플 수: 310


In [10]:
# Colab 셀 7: 평가 지표 & compute_metrics 정의

import numpy as np

# 평가 지표 로드
bleu_metric  = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

# metrics 함수 정의
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # -100 → pad_token_id
    preds  = np.where(preds  == -100, tokenizer.pad_token_id, preds)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    # 디코딩
    decoded_preds  = tokenizer.batch_decode(preds,  skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # BLEU & ROUGE-L 계산
    bleu  = bleu_metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])["bleu"]
    rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)["rougeL"]
    return {"bleu": bleu, "rougeL": rouge}

print("compute_metrics 함수가 준비되었습니다.")

compute_metrics 함수가 준비되었습니다.


In [14]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/kot5-rewrite",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=64,
    generation_num_beams=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=100,
    save_total_limit=2,
    lr_scheduler_type="linear",
    logging_dir="/content/logs",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # 여전히 BLEU·ROUGE 함수는 넘겨줍니다
)

print("Trainer 준비 완료.")

Trainer 준비 완료.


/tmp/ipython-input-1145846697.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [15]:
# Colab 셀 9 (수정): 학습 시작 → 수동 평가
trainer.train()

# 학습 끝나면 한 번만 BLEU/ROUGE 평가
metrics = trainer.evaluate()
print("최종 평가 지표:", metrics)

# 모델·토크나이저 저장
trainer.save_model("/content/kot5-rewrite")
tokenizer.save_pretrained("/content/kot5-rewrite")

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
500,5.276300
1000,4.487300
1500,3.994600
2000,3.756200


최종 평가 지표: {'eval_loss': 3.705479860305786, 'eval_bleu': 0.006509163916181852, 'eval_rougeL': 0.008602150537634409, 'eval_runtime': 47.3459, 'eval_samples_per_second': 6.548, 'eval_steps_per_second': 1.647, 'epoch': 3.0}


('/content/kot5-rewrite/tokenizer_config.json',
 '/content/kot5-rewrite/special_tokens_map.json',
 '/content/kot5-rewrite/tokenizer.json')

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1) 디바이스 및 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FINETUNED_DIR = "/content/kot5-rewrite"   # 실제 모델 경로로 수정
tokenizer = AutoTokenizer.from_pretrained(FINETUNED_DIR)
model     = AutoModelForSeq2SeqLM.from_pretrained(FINETUNED_DIR).to(device)

# 2) 입력 받기
print("대화 맥락을 줄 단위로 입력하세요. 마지막에 빈 줄(Enter) 한 번 더 치면 입력이 종료됩니다.")
lines = []
while True:
    line = input()
    if not line.strip():
        break
    lines.append(line.strip())

if len(lines) < 2:
    raise ValueError("최소 2줄 이상의 대화(맥락+순화 대상)가 필요합니다.")

# 3) 포맷팅 (Step 1 영어 프롬프트 기준)
prefix = "rewrite last line politely:"
input_text = " ".join([prefix] + lines)

# 4) 생성
inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)
outputs = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 5) 결과 출력
print("\n=== 입력 대화 ===")
for l in lines:
    print(l)
print("\n=== 순화 결과 ===")
print(result)

대화 맥락을 줄 단위로 입력하세요. 마지막에 빈 줄(Enter) 한 번 더 치면 입력이 종료됩니다.
A: 오늘 점심 메뉴 추천 좀
B: 음... 닭갈비?
A: 닭갈비는 좀 별론데...
B: 아 개까다롭네 진짜 미친



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



=== 입력 대화 ===
A: 오늘 점심 메뉴 추천 좀
B: 음... 닭갈비?
A: 닭갈비는 좀 별론데...
B: 아 개까다롭네 진짜 미친

=== 순화 결과 ===
닭갈비는 좀 별미인 것 같아
